In [1]:
# Set root dir
import os
os.chdir('..')
print(os.getcwd())

/Users/tonpoppe/workspace/tigger_adj_rep/tigger_adj


In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import tensorflow as tf
from pyspark.sql.window import Window
from pyspark.sql import functions as F
import yaml


In [3]:
from GAE.graph_case_controller import GraphAutoEncoder
from GAE.data_feeder_graphframes import DataFeederGraphFrames
from tigger_package.orchestrator import Orchestrator
from tigger_package.label_transfer import LabelTransferrer

loaded


In [4]:
conf = SparkConf().setAppName('appName').setMaster('local')
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

23/12/31 09:00:26 WARN Utils: Your hostname, MacBook-Pro-2.local resolves to a loopback address: 127.0.0.1; using 10.100.81.74 instead (on interface en0)
23/12/31 09:00:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/31 09:00:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [10]:
config_dict = yaml.safe_load('''
    weight_col: 'edge_att0'
    label_col: none
    support_size: [4, 4]
    dims: [2, 6, 6, 6]
    batch_size: 64
    hub0_dim: 6
    seed: 1
    learning_rate: 0.001
    usebn: true
    verbose: true
    epochs: 3
''')

In [11]:
folder = "data/erdos/"
orchestrator = Orchestrator(folder)
nodes = orchestrator._load_nodes()
edges = orchestrator._load_edges()                                 

In [12]:
lt = LabelTransferrer(
    nodes=nodes, 
    edges=edges, 
    synth_nodes=nodes, 
    synth_edges=edges,
    config_dict=config_dict,
    spark=spark)

In [13]:
gae = lt.transfer()

/Users/tonpoppe/anaconda3/envs/tg_geo/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


The following node labels are included ['attr0', 'attr1', 'attr2', 'attr3', 'attr4', 'attr5', 'attr6', 'attr7']
The following edge labels are included ['weight', 'edge_att1', 'edge_att2', 'edge_att3', 'edge_att4', 'edge_att5', 'edge_att6']


train nodes [399 386 575 684 163  43 278 616 132 774] ...
val nodes [312 980 209  55 342  49 707 687 857 669] ...


In [14]:
history = gae.fit(epochs=3, layer_wise=False)

train nodes [399 386 575 684 163  43 278 616 132 774] ...
val nodes [312 980 209  55 342  49 707 687 857 669] ...
Epoch 1/3
10/10 [==============================] - 1s 34ms/step - loss: 0.3499 - output_1_loss: 0.2483 - output_2_loss: 0.1017 - val_loss: 0.3901 - val_output_1_loss: 0.2718 - val_output_2_loss: 0.1183
Epoch 2/3
10/10 [==============================] - 0s 9ms/step - loss: 0.3154 - output_1_loss: 0.2220 - output_2_loss: 0.0934 - val_loss: 0.3659 - val_output_1_loss: 0.2536 - val_output_2_loss: 0.1122
Epoch 3/3
10/10 [==============================] - 0s 9ms/step - loss: 0.2950 - output_1_loss: 0.2065 - output_2_loss: 0.0886 - val_loss: 0.3395 - val_output_1_loss: 0.2332 - val_output_2_loss: 0.1064


In [15]:
e = gae.calculate_embeddings(graph)
e

NameError: name 'graph' is not defined